In [2]:
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\\oracle\\instantclient_12_1")
import json

In [3]:
import pandas as pd

In [4]:
pw = open("C:\\Users\\aneslin\\Documents\\python\\item-remap\\passwords.json")
x = json.load(pw)
x

{'user': '5C_READ',
 'password': 'odbc201009',
 'server': 'FCAA.LIBRARY.UMASS.EDU:1521/ALEPH22P'}

In [5]:
pool = cx_Oracle.connect(user=x["user"], password = x["password"], dsn = x["server"])

In [6]:
query = ''' 
select
'ac' as prefix,
substr(ord.Z68_REC_KEY,0,9),
ord.Z68_ORDER_Type ,
rr.z13key
,
 ord.Z68_Order_NUMBER,
 ord.Z68_ORDER_NUMBER_1,
 ord.Z68_ORDER_NUMBER_2,
ord.Z68_ORDER_GROUP,
ord.Z68_LIBRARY_NOTE,
ord.z68_e_note,
ord.Z68_OPEN_DATE,
ord.z68_ORDER_DATE,
ord.Z68_ORDER_STATUS,
ord.z68_invoice_status,
ord.Z68_ORDER_STATUS ,
ord.Z68_ORDER_STATUS_DATE_X,
ord.z68_method_of_aquisition,
concat('AC' ,ord.Z68_VENDOR_CODE) as vendorCode,
budName.fundName,
ord.z68_vendor_note,
ord.Z68_NO_UNITS,
ord.Z68_UNIT_PRICE,
ord.Z68_TARGeT_TEXT,
ord.z68_subscription_date_from,
ord.z68_subscription_date_to,
ord.Z68_SUBSCRIPTION_RENEW_DATE,
rr.Z13_TITLE,
rr.author,
ord.z68_isbn,
ord.z68_material_type,
ord.z68_vendor_reference_no
-- Umass and Smith will use the same export critera as Amherst. To run the Umass query, change the AMH50.Z68 and AMH50.Z103 to UMA50.Z68 and UMA.Z103.
-- you will also need to change the org code prefix to UM or SC
from AMH50.Z68 ord
LEFT join
(
    select brief.Z13_REC_KEY as z13key , brief.Z13_TITLE,brief.Z13_author as author ,substr(lkr.Z103_REC_KEY,6,9) as ADM_N from
    AMH50.Z103 lkr
    inner join
    FCL01.z13 brief
    on SUBSTR(brief.Z13_REC_KEY, 0,9) = substr(lkr.Z103_REC_KEY_1,6,9)
    and lkr.Z103_LKR_TYPE='ADM'
    and substr(lkr.Z103_REC_KEY_1,1,5)='FCL01'
) rr
on substr(ord.Z68_REC_KEY ,0, 9) = rr.ADM_N
left join
( 
Select ordnum, substr(Z601_REC_KEY ,0, 50) as fundName from 
    (
    select Z601_REC_KEY, substr(Z601_REC_KEY_3,0,9) as ordnum, max(substr(Z601_REC_KEY,50,25)) over (partition by substr(Z601_REC_KEY_3,0,9)) as max_open_date from AMH50.Z601 where substr(Z601_REC_KEY_3,0,9)<>'000000000'
    ) 
    where max_open_date=substr(Z601_REC_KEY ,50,25)
) budName
on rr.ADM_N = budname.ordnum
WHERE
(ord.Z68_ORDER_TYPE = 'S'
and ord.Z68_ORDER_STATUS = 'SV'
)
OR
(ord.Z68_ORDER_TYPE = 'O'
and ord.Z68_ORDER_STATUS = 'SV')
OR
(ord.Z68_ORDER_TYPE = 'M'
and ord.Z68_ORDER_STATUS = 'SV'
and ord.Z68_OPEN_DATE > 20190701)
'''

In [7]:
cursor = pool.cursor()

In [8]:
cursor.execute(query)

<cx_Oracle.Cursor on <cx_Oracle.Connection to 5C_READ@FCAA.LIBRARY.UMASS.EDU:1521/ALEPH22P>>

In [9]:
def makeDictFactory(cursor):
    columnNames = [d[0] for d in cursor.description]
    def createRow(*args):
        return dict(zip(columnNames, args))
    return createRow

In [10]:
cursor.rowfactory = makeDictFactory(cursor)

In [11]:
x = cursor.fetchall()

In [12]:
df = pd.DataFrame(x)

In [13]:
df['Z13KEY']= df['Z13KEY'].str.strip()

In [14]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2023 entries, 0 to 2022
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   PREFIX                       2023 non-null   object
 1   SUBSTR(ORD.Z68_REC_KEY,0,9)  2023 non-null   object
 2   Z68_ORDER_TYPE               2023 non-null   object
 3   Z13KEY                       2023 non-null   object
 4   Z68_ORDER_NUMBER             2023 non-null   object
 5   Z68_ORDER_NUMBER_1           221 non-null    object
 6   Z68_ORDER_NUMBER_2           11 non-null     object
 7   Z68_ORDER_GROUP              320 non-null    object
 8   Z68_LIBRARY_NOTE             1816 non-null   object
 9   Z68_E_NOTE                   67 non-null     object
 10  Z68_OPEN_DATE                2023 non-null   int64 
 11  Z68_ORDER_DATE               2023 non-null   int64 
 12  Z68_ORDER_STATUS             2023 non-null   object
 13  Z68_INVOICE_STATUS           1897

In [19]:
cols = ['Z68_LIBRARY_NOTE', 'Z68_ORDER_GROUP','FUNDNAME']
df['notes'] = df[cols].agg("; ".join, axis = 1)

TypeError: sequence item 1: expected str instance, NoneType found

In [16]:
df2 = df[['PREFIX','Z68_ORDER_NUMBER', 'VENDORCODE', 'Z68_ORDER_TYPE', "Z68_ORDER_STATUS", "Z68_METHOD_OF_AQUISITION" ]]

SyntaxError: invalid syntax (Temp/ipykernel_7444/755270441.py, line 1)

In [ ]:
df2.rename(columns={'PREFIX': 'poNumberPrefix',
'Z68_ORDER_NUMBER': 'poNumber',
'Z68_ORDER_TYPE': 'orderType',
"Z68_ORDER_STATUS": "workflowStatus",
"Z68_METHOD_OF_AQUISITION": "acquisitionMethod"   
   
    }, inplace=True )

In [ ]:
df2.head()